## Load libraries

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

from alphai_watson.transformer import NullTransformer

from alphai_rickandmorty_oracle.datasource.kddcup99 import KDDCup99DataSource
from alphai_rickandmorty_oracle.detective import RickAndMortyDetective

/opt/anaconda/envs/ai/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/anaconda/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Enabling weight norm
Uppercase local vars:
	BATCH_SIZE: 50
	CRITIC_ITERS: 5
	DEFAULT_FIT_EPOCHS: 1000
	DEFAULT_LEARN_RATE: 0.0001
	DEFAULT_TRAIN_ITERS: 5000
	DEFAULT_Z_DIM: 32
	DIAGNOSIS_LEARN_RATE: 0.01
	DIM: 64
	DISC_FILTER_SIZE: 5
	INIT_KERNEL: <function variance_scaling_initializer.<locals>._initializer at 0x7f11ef276488>
	LAMBDA: 10
	LAMBDA_2: 2.0
	OUTPUT_DIM: 121


## Define KDDCup99 Datasource

In [2]:
file_path = '../../tests/resources'
data_filename = os.path.join(file_path, 'kddcup.data_10_percent_corrected')
header_filename = os.path.join(file_path, 'kddcup.names')

kdd_datasource = KDDCup99DataSource(source_file=data_filename,
                                    header_file=header_filename,
                                    transformer=NullTransformer(8, 8))

DEBUG:root:Start file parsing.
DEBUG:root:Normal (97278, 122); Train (68095, 121); Test(29183, 121)
DEBUG:root:Abnormal (396743, 121)
DEBUG:root:End file parsing.


In [3]:
data_normal_train = kdd_datasource.get_train_data('NORMAL')
data_normal_test = kdd_datasource.get_train_data('NORMAL_TEST')
data_abnormal_test = kdd_datasource.get_train_data('ABNORMAL_TEST')

## Define Model

In [4]:
model_dir = './kddcup99_models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

detective = RickAndMortyDetective(model_configuration={
    'batch_size': 64,
    'output_dimensions': 121,
    'train_iters': 1000,
    'save_path' : '{}/KDDCup99-model'.format(model_dir)
})

detective.train(data_normal_train)

DEBUG:root:Starting session
DEBUG:root:Start training loop...
INFO:root:Initialising Model
INFO:root:Training iteration 0 of 1000
INFO:root:Training iteration 100 of 1000
INFO:root:Training iteration 200 of 1000
INFO:root:Training iteration 300 of 1000
INFO:root:Training iteration 400 of 1000
INFO:root:Training iteration 500 of 1000
INFO:root:Training iteration 600 of 1000
INFO:root:Training iteration 700 of 1000
INFO:root:Training iteration 800 of 1000
INFO:root:Training iteration 900 of 1000
DEBUG:root:Training complete.


## Evaluate results

#### Train data (Normal)

In [5]:
def model_accuracy(data, status, mean=None):
    eval = detective.detect(data).data * -1
    if mean is None:
        mean = np.median(eval)
    ground_truth = [status] * len(eval)
    prediction = [1 if x >= mean else 0 for x in eval]
    print('Accuracy: {0:.2f}%'.format(100*accuracy_score(ground_truth, prediction)))
    return mean

In [6]:
mean = model_accuracy(data_normal_train, 1)

INFO:root:Running detector on <alphai_watson.datasource.Sample object at 0x7f11a8c9c2b0>
INFO:root:Detection completed in 1.076493689790368


Accuracy: 50.00%


#### Test data (Normal)

In [7]:
_ = model_accuracy(data_normal_test, 1, mean)

INFO:root:Running detector on <alphai_watson.datasource.Sample object at 0x7f11a8c9c278>
INFO:root:Detection completed in 0.4512240421026945


Accuracy: 49.55%


#### Test data (Abnormal)

In [8]:
_ = model_accuracy(data_abnormal_test, 0, mean)

INFO:root:Running detector on <alphai_watson.datasource.Sample object at 0x7f11a8c9c2e8>
INFO:root:Detection completed in 5.190419405698776


Accuracy: 76.86%
